In [1]:
import numpy as np
import csv

# Predict via the user-specific median.
# If the user has no data, use the global median.

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'

# Load the training data.
train_data = {}
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)
    for row in train_csv:
        user   = row[0]
        artist = row[1]
        plays  = row[2]
    
        if not user in train_data:
            train_data[user] = {}
        
        train_data[user][artist] = int(plays)

# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

# Write out test solutions.
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]

            if user in user_medians:
                soln_csv.writerow([id, user_medians[user]])
            else:
                print "User", id, "not in training data."
                soln_csv.writerow([id, global_median])

In [2]:
import pandas as pd

In [4]:
df = pd.DataFrame.from_csv('soln_csv')

IOError: File soln_csv does not exist

In [28]:
import pandas as pd
df = pd.read_pickle('profiles.pkl')

In [29]:
df.head()

,user,sex,age,country,latitudes,longitudes,gdp
0,fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d,0,25,Sweden,59.3327,18.06450,44033.943287
1,5909125332c108365a26ccf0ee62636eee08215c,1,29,Iceland,64.1353,-21.89520,41236.473229
2,d1867cbda35e0d48e9a8390d9f5e079c9d99ea96,1,30,United States,38.8895,-77.03200,52117.745853
3,63268cce0d68127729890c1691f62d5be5abd87c,1,21,Germany,52.5235,13.41150,43602.359247
4,02871cd952d607ba69b64e2e107773012c708113,1,24,Netherlands,52.3738,4.89095,45691.392933


In [30]:
import pandas as pd

df = pd.io.parsers.read_csv(
    'https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/wine_data.csv',
    header=None,
    )

In [32]:
df.shape

(178, 14)

In [47]:
from sklearn.cross_validation import train_test_split

X_wine = df.values[:,1:]
y_wine = df.values[:,0]

X_train, X_test, y_train, y_test = train_test_split(X_wine, y_wine,
    test_size=0.30, random_state=12345)

In [48]:
X_wine

array([[  1.42300000e+01,   1.71000000e+00,   2.43000000e+00, ...,
          1.04000000e+00,   3.92000000e+00,   1.06500000e+03],
       [  1.32000000e+01,   1.78000000e+00,   2.14000000e+00, ...,
          1.05000000e+00,   3.40000000e+00,   1.05000000e+03],
       [  1.31600000e+01,   2.36000000e+00,   2.67000000e+00, ...,
          1.03000000e+00,   3.17000000e+00,   1.18500000e+03],
       ..., 
       [  1.32700000e+01,   4.28000000e+00,   2.26000000e+00, ...,
          5.90000000e-01,   1.56000000e+00,   8.35000000e+02],
       [  1.31700000e+01,   2.59000000e+00,   2.37000000e+00, ...,
          6.00000000e-01,   1.62000000e+00,   8.40000000e+02],
       [  1.41300000e+01,   4.10000000e+00,   2.74000000e+00, ...,
          6.10000000e-01,   1.60000000e+00,   5.60000000e+02]])

In [49]:
from sklearn import preprocessing

std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

In [50]:
X_train_std

array([[-1.87897935,  1.16139809, -1.95836639, ..., -0.97692884,
         0.30321156, -0.54967345],
       [-0.75525649, -1.08867904, -0.91454568, ...,  0.28222389,
         0.24180162, -0.25245748],
       [-0.36010119, -0.7010253 , -0.3180767 , ..., -0.06512859,
        -0.84822474, -0.75287213],
       ..., 
       [ 0.98589652,  0.33552709, -0.16895946, ..., -1.19402414,
        -1.43161913, -0.3889342 ],
       [-1.16276038,  1.63332438,  0.12927503, ..., -0.8032526 ,
        -0.06524806, -0.49508276],
       [ 1.28226299, -0.57461647, -0.50447326, ...,  0.28222389,
         1.50070531,  0.88484854]])

In [51]:
X_train_std.shape

(124, 13)

In [52]:
from sklearn.decomposition import PCA

# on non-standardized data
pca = PCA(n_components=2).fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)


# om standardized data
pca_std = PCA(n_components=2).fit(X_train_std)
X_train_std = pca_std.transform(X_train_std)
X_test_std = pca_std.transform(X_test_std)

In [53]:
X_train_std.shape

(124, 2)